## Chap 7 Regularization for Deep Learning
We defined regularization as __any modification we make to a learning algorithm that is intended to reduce its generalization error but not its training error.__

In the context of deep learning, most regularization strategies are based on regularizing estimators. Regularization of an estimator works by trading increased bias for reduced variance.

Deep learning algorithms are typically applied to extremely complicated domains such as images, audio sequences and text, for which the true generation process essentially involves simulating the entire universe. To some extent, we are always trying to fit a square peg (the data generating process) into a round hole (our model family).

What this means is that __controlling the complexity of the model__ is NOT a simple matter of finding the __model of the right size__, with the right number of parameters. Instead, we might find and -- indeed in practical deep learning scenarios, we almost always do find -- that the __best fitting model__ (in the sense of minimizing generalization error) is a __large model that has been regularized appropriately.__

### 7.1 Parameter Norm Penalties
Many regularization approaches are based on limiting the capacity of models by adding a parameter norm penalty $\Omega(\theta)$ to the objective function $J$. We denote the regularized objective function by $\tilde J$

$$\tilde J(\theta; X,y) = J(\theta; X,y) + \alpha \Omega(\theta)$$

For neural networks, we typically choose to use a parameter norm penalty that penalizes only the weights of the affine transformation at each layer and leaves the biases unregularized. The biases typically require less data to fit accurately than the weights.

Each weight specifies how two variables interact. Fitting the weight well requires observing both variables in a variety of conditions. Each __bias controls only a single variable.__ This means that __we do NOT induce too much variance by leaving the biases unregularized.__ Also, __regularizing the bias parameters can introduce a significant amount of underfitting.__

In the context of neural networks, it is sometimes desirable to __use a separate penalty with a different α coefficient for each layer of the network.__ Because it can be expensive to search for the correct value of multiple hyperparameters, it is still reasonable to use the same weight decay at all layers just to __reduce the size of search space.__

#### 7.1.1 $L^2$ Parameter Regularization
L2 parameter norm penalty commonly known as __weight decay__ This regularization strategy __drives the weights closer to the origin__ by adding a regularization term $\Omega(\theta) = \frac{1}{2} ||w||^2$ to the objective function.

We can gain some insight into the behavior of weight decay regularization by studying the gradient of the regularized objective function. To simplify the presentation, we assume no bias parameter, so $\theta$ is just w. 

__Objective function:__ 
$$\tilde J(w) = \frac{\alpha}{2} w^T w + J(w)$$

__Gradient:__ 
$$\nabla_w \tilde J(w) = \alpha w + \nabla_w J(w)$$

__Weight update:__ 
$$w \leftarrow w - \epsilon  (\alpha w + \nabla_w J(w))$$

$$w \leftarrow (1- \epsilon \alpha)w - \epsilon \nabla_w J(w)$$

We can see that the addition of the weight decay term has modified the learning rule to multiplicatively shrink the weight vector by a __constant factor__ on each step, just before performing the usual gradient update. 

We will further simplify the analysis by making a __quadratic approximation__ to the objective function in the neighborhood of the value of the weights that obtains minimal unregularized training cost, $w^∗ = \arg \min_w J(w)$ 

> 為了簡化模型，我們將目標函數在最低點做二階的泰勒展開式作為近似。

Taylor series expalsion around $w^*$
\begin{align}
J(w) & \approx J(w^*) + (w-w^*)^T g + \frac{1}{2} (w-w^*)^T H (w-w^*) \\
&( \because w^* = arg \min_w J(w) \therefore g = 0) \\
\hat J(w) & = J(w^{*}) + \frac{1}{2}(w-w^*)^T H(w-w^*) \\
\end{align}

> $\hat J(w)為J(w)在w^*附近之近似，之後的討論中J(w)都用\hat J(w)代替$

where H is the Hessian matrix of J with respect to w evaluated at $w^*$. Because $w^∗$ is the location of a minimum of J, we can conclude that __H is positive semidefinite.__

The minimum of J occurs where its gradient
$$\nabla_w J(w) \approx \nabla_w \hat J(w) = H(w-w^*)$$
is equal to zero. 

We can now solve for the regularized version of J. We use the variable $\tilde w = arg \min_w \tilde J(w)$ to represent the location of the minimum of $\tilde J$. 

\begin{align}
\because \tilde J(w) &= \frac{\alpha}{2} w^T w + J(w) \approx \frac{\alpha}{2} w^T w + \hat J(w) \\
\therefore \nabla_w \tilde J(w) &= \alpha w + \nabla_w \hat J(w) \\
                                &= \alpha w + H(w-w^*)           \\
\nabla_w \tilde J(w) &= 0 \\
                     &\Rightarrow \alpha \tilde w + H(\tilde w - w^*) = 0 \\
                     &\Rightarrow (H+\alpha I)\tilde w = H w^*            \\
                     &\Rightarrow \tilde w = (H+\alpha I)^{-1} H w^*      \\ 
\end{align}

As $\alpha$ approaches 0, the regularized solution $\tilde w$ approaches $w^*$

\begin{align}
w^∗ &= \arg \min_w J(w)        \\ 
\tilde w &= \arg \min_w \tilde J(w) \\
\end{align}

> $w^*$是沒有正則化的最佳解，$\tilde w$是正則化的最佳解 

Because H is __real and symmetric__, we can decompose it into a diagonal matrix D and an orthonormal basis of eigenvectors, Q, such that $H = QDQ^T, QQ^T=Q^TQ=I$

\begin{align}
\tilde w &= (QDQ^T + \alpha I)^{-1} QDQ^T w^* \\ 
&= (QDQ^T + Q \alpha Q^T)^{-1} QDQ^T w^* \\
&= \left[ Q(D+\alpha I)Q^T \right]^{-1} QDQ^T w^* \\
&= Q(D+\alpha I)^{-1} DQ^T w^* \\
\end{align}

We see that the effect of weight decay is to rescale $w^*$ along the axes defined by the eigenvectors of H.

Specifically, the component of $w^∗$ that is aligned with the i-th eigenvector of H is rescaled by a factor of $\frac{\lambda_i}{\lambda_i + \alpha}$

Along the directions where the eigenvalues of H are relatively large, the effect of regularization is relatively small. On the other hand, the effect of regularization will shrink eigenvalues to zero.

\begin{align}
\lambda_i >> \alpha: & \\
& \lambda_i' = \lambda_i \cdot \frac{\lambda_i}{\lambda_i + \alpha} \approx \lambda_i \\
\lambda_i << \alpha: & \\
& \lambda_i' = \lambda_i \cdot  \frac{\lambda_i}{\lambda_i + \alpha} \approx 0 
\end{align}

![L2 norm illustration](ref/Fig7.1.png)

__Linear Regression example with L2 norm__ 

__Cost function:__

\begin{align}
J(w) &= (Xw-y)^T (Xw-y) \\ 
\tilde J(w) &= (Xw-y)^T(Xw-y) + \frac{1}{2} \alpha w^T w \\
\end{align}

__Unregularized Solution:__
\begin{align}
\nabla_w J(w) &= 2 X^T(Xw-y) = 0 \\
  \Rightarrow &= X^T X w - X^Ty = 0 \\
  \Rightarrow w &= (X^T X)^{-1} X^T y \\
\end{align}

__Regularized Solution:__
\begin{align}
\nabla_w \tilde J(w) &= 2 X^T(Xw-y) + \alpha w = 0 \\
  \Rightarrow &= 2X^T X w - 2X^Ty + \alpha w = 0 \\
  \Rightarrow \hat w &= \left( X^T X + \frac{\alpha}{2} I \right)^{-1} X^T y \\
\end{align}

__Analysis:__ 

The matrix $X^TX$ is proportional to the covariance matrix $\frac{1}{m}X^TX$. Using $L^2$ regularization replaces this matrix with $\left( X^TX+\alpha I \right)^{-1}$. The new matrix is the same as the original one, but with the addition of α to the diagonal. __The diagonal entries of this matrix correspond to the variance of each input feature.__ We can see that L2 regularization causes the learning algorithm to “perceive” the input X as having __higher variance__, which makes it __shrink the weights on features whose covariance with the output target is low compared to this added variance.__

__Example:__

Let $X^TX = \begin{bmatrix}
10  &  x   & x   \\
x   &  0.1 & x   \\
x   &  x   & 0.2 \\
\end{bmatrix}$, $cov(x_1, x_1) = 100 \cdot cov(x_2, x_2)$. Assume $\alpha = 10000$, then the L2 norm will see $X^TX$ as $(X^TX)' = \begin{bmatrix}
10010 &  x       & x       \\
x     &  10000.1 & x       \\
x     &  x       & 10000.2 \\
\end{bmatrix}$, thus, $cov(x_1, x_1) \approx cov(x_2, x_2) = 10000$ 

#### 7.1.2 L1 Regularization
$$\Omega(\theta) = ||w||_1 = \sum_i |w_i|$$

__Objective function:__
$$\tilde J(w) = \alpha ||w||_1 + J(w)$$
__Gradient:__
\begin{align} 
\nabla_w \tilde J(w) &= \alpha \cdot sign(w) + \nabla_w J(w) \\
& \approx \alpha \cdot sign(w) + \nabla_w \hat J(w) \\
        &= \alpha \cdot sign(w) + H(w - w^*)         \\
\end{align}

The regularization contribution to the gradient no longer scales linearly with each wi; instead it is a __constant factor__ with a sign equal to $sign(w_i)$. One consequence of this form of the gradient is that we will not necessarily see clean algebraic solutions to quadratic approximations of J(w) as we did for L2 regularization.

Because the L1 penalty does not admit clean algebraic expressions in the case of a fully general Hessian, we will also make the further simplifying assumption that the __Hessian is diagonal__, $H = diag([H_{1,1},...,H_{n,n}])$, where each $H_{i,i} > 0$.
This assumption holds if the data for the linear regression problem has been preprocessed to __remove all correlation between the input features__, which may be accomplished using PCA.

Under the assumption that Hessian is diagonal, the Taylor expansion of $J(w)$ around $w^*$ is as follow, 
\begin{align}
\hat J(w) &= J(w^*) + \frac{1}{2} (w-w^*)^T H (w-w^*) \\
&= J(w^*) + \sum_i \left[ \frac{1}{2} H_{i,i} (w_i - w_{i}^{*})^2 \right] \\
\tilde J(w) &= J(w) + \alpha |w| \\
            &\approx \hat J(w) + \alpha |w| \\ 
            &= J(w^*) + \sum_i \left[ \frac{1}{2} H_{i,i} (w_i - w_{i}^{*})^2 + \alpha |w_i| \right] \\ 
\end{align}


The problem of minimizing this approximate cost function has an analytical solution (for each dimension i), with the following form: 
\begin{align}
\nabla_w \tilde J(w) &= \alpha \cdot sign(w) + \nabla_w J(w) \\
&= \alpha \cdot sign(w) + \nabla_w \hat J(w) \\
&= \alpha \cdot sign(w) + H(w-w^*) = \vec 0\\ 
\because &\text{Assume H is a diagonal matrix} \\ 
\therefore &\forall{i=1 \sim n} \\ 
\Rightarrow & \alpha \cdot sign(w_i) + H_{i,i} \cdot (w_i - w_i^*) = 0 \\
\Rightarrow & \tilde w_i = sign(w_i^{*}) \cdot max \left\{ |w_i^*|- \frac{\alpha}{H_{i,i}} , 0 \right\} \\
\end{align}

__Analysis:__

Solve for $w_i$ in the equation: $\alpha \cdot sign(w_i) + H_{i,i} \cdot (w_i - w_i^*) = 0$

We first start with discussing the sign of $w_i$ 

\begin{align} 
w_i &=\left\{
\begin{aligned}
& w_i^* - \frac{\alpha}{H_{i,i}}, (w_i > 0) \\
& w_i^* + \frac{\alpha}{H_{i,i}}, (w_i < 0) \\
& 0, (w_i = 0) \\
\end{aligned} 
\right. 
\end{align} 

Then we discuss all possible values for $w_i^*$

Case 1: $0<w_i^*<\frac{\alpha}{H_{i,i}}$
\begin{align} 
0<w_i^*<\frac{\alpha}{H_{i,i}}&\left\{
\begin{aligned}
(w_i > 0), \; & w_i^* - \frac{\alpha}{H_{i,i}} = w_i <0  \Rightarrow\!\Leftarrow \\
(w_i < 0), \; & w_i^* + \frac{\alpha}{H_{i,i}} = w_i >0 \Rightarrow\!\Leftarrow \\
(w_i = 0) \; &\\
\end{aligned} 
\right. 
\end{align} 

Thus, $0<w_i^*<\frac{\alpha}{H_{i,i}} \Rightarrow \tilde w_i = 0$

Case 2: $w_i^*>\frac{\alpha}{H_{i,i}}$
\begin{align} 
w_i^*>\frac{\alpha}{H_{i,i}}&\left\{
\begin{aligned}
(w_i > 0), \; & w_i^* - \frac{\alpha}{H_{i,i}} = w_i >0 \\
(w_i < 0), \; & w_i^* + \frac{\alpha}{H_{i,i}} = w_i >0 \Rightarrow\!\Leftarrow \\
\end{aligned} 
\right. 
\end{align} 

Thus, $w_i^*>\frac{\alpha}{H_{i,i}} \Rightarrow \tilde w_i = w_i^* - \frac{\alpha}{H_{i,i}}$

Case 3: $-\frac{\alpha}{H_{i,i}}<w_i^*<0$
\begin{align} 
-\frac{\alpha}{H_{i,i}}<w_i^*<0 &\left\{
\begin{aligned}
(w_i > 0), \; & w_i^* - \frac{\alpha}{H_{i,i}} = w_i <0  \Rightarrow\!\Leftarrow \\
(w_i < 0), \; & w_i^* + \frac{\alpha}{H_{i,i}} = w_i >0 \Rightarrow\!\Leftarrow \\
\end{aligned} 
\right. 
\end{align} 

Thus, $-\frac{\alpha}{H_{i,i}}<w_i^*<0 \Rightarrow \tilde w_i = 0$

Case 4: $w_i^* < -\frac{\alpha}{H_{i,i}}$
\begin{align} 
w_i^* < -\frac{\alpha}{H_{i,i}}&\left\{
\begin{aligned}
(w_i > 0), \; & w_i^* - \frac{\alpha}{H_{i,i}} = w_i < 0 \Rightarrow\!\Leftarrow \\
(w_i < 0), \; & w_i^* + \frac{\alpha}{H_{i,i}} = w_i < 0 \\ 
\end{aligned} 
\right. 
\end{align} 

Thus, $w_i^* <-\frac{\alpha}{H_{i,i}} \Rightarrow \tilde w_i = w_i^* + \frac{\alpha}{H_{i,i}}$

__Observation:__

For case 1 and 3, the optimal value of $\tilde w_i = 0$ This occurs because the contribution of J(w) to the regularized objective $\tilde J(w)$ is overwhelmed in direction i by the L1 regularization which pushes the value of $w_i$ to zero.

For case 2 and 4, the regularization doesn't move the optimal value of $w_i$ to zero but instead it just shifts it in that direction by a distance equal to $\frac{\alpha}{H_{i,i}}$

In conclusion, the solution to \\[\alpha \cdot sign(w_i) + H_{i,i} \cdot (w_i - w_i^*) = 0\\] is \\[\tilde w_i = sign(w_i^{*}) \cdot max \left\{ |w_i^*|- \frac{\alpha}{H_{i,i}} , 0 \right\}\\]

In comparison to L2 regularization, L1 regularization results in a solution that is more sparse. Sparsity in this context refers to the fact that some parameters have an optimal value of zero. The sparsity of L1 regularization is a qualitatively different behavior than arises with L2 regularization.

If we revisit that equation using the assumption of a diagonal and positive definite Hessian H that we introduced for our analysis of L1 regularization, we find that $\tilde w_i = \frac{H_{i,i}}{H_{i,i} + \alpha} w_i^*$ This demonstrates that L2 regularization doesn't cause the parameters to become sparse while L1 regularization may do so for large enough $\alpha$. 

In the following section, detailed clarification is made for both L1 and L2 regularization. 
- L1 regularization:

H is the diagonal Hessian matrix around point $w^*$. For each diagonal element (Let's say the i-th element) in H, L1 regularization makes the following correction to the unregularized solution $w^*$ 

\\[\tilde w_i = \frac{H_{i,i}}{H_{i,i} + \alpha} w_i^*\\]

> L1 scales the diagonal matrix H directly.

- L2 regularization: 

H is the Hessian matrix around point $w^*$. By orthornomally diagonized H, L2 regularization rescales $w^*$ in every direction of eigenvector of H. 

\\[\tilde w = Q(D+\alpha I)^{-1} DQ^T w^*, \: H=QDQ^T\\]

> L2 scales the eigenvalues of H, scaled value is less unlikely to become a zero matrix.  

The sparsity property induced by L1 regularization has been used extensively as a __feature selection__ mechanism. Feature selection simplifies a machine learning problem by choosing which subset of the available features should be used.

The L1 penalty causes a subset of the weights to become zero, suggesting that the corresponding features may safely be discarded.

### 7.2 Norm Penalties as Constrained Optimization
__Norm Penalties__

The cost function regularized by a parameter norm penalty: 
\\[\tilde J(\theta) = J(\theta) + \alpha \Omega(\theta)\\]

Suppose we want to minimize the cost function \\[f(w)= ||Xw-y||^2_2 = (Xw-y)^T(Xw-y)\\] with L2 regularization. The regularized cost function become \\[\tilde f(w) = (Xw-y)^T(Xw-y) + \frac{1}{2} \alpha w^Tw\\] We then solve the derivative function to obtain $\tilde w = \arg \min_w f(w)$

\begin{align}
\nabla_w \tilde f(w) &= 2X^TXw - 2 X^Ty + \alpha w = 0 \\
\tilde w &= (X^TX + \frac{\alpha}{2} I)^{-1} X^T y \\  
\end{align}

__Constrained Optimization__ 

We can minimize a function subject to constraints by constructing a generalized Lagrange function, consisting of the original objective function plus a set of penalties. Each penalty is a product between a coefficient, called a __Karush–Kuhn–Tucker (KKT) multiplier__, and a function representing
whether the constraint is satisfied. 

Suppose we want to minimize the cost function \\[f(w)=\frac{1}{2} ||Xw-y||^2_2\\] subject to $w^Tw \leq 1$. To do so, we introduce the Lagrangian \\[L(w, \lambda) = f(w) + \lambda(w^Tw-1)\\] and then solve the equation \\[\min_w \max_{\lambda, \lambda \geq 0} L(w, \lambda)\\] We then solve the derivative function to obtain $\tilde w = \arg \min_w L(w, \lambda)$

\begin{align}
\nabla_w L(w, \lambda) &= X^TXw - X^Ty + 2 \lambda w = 0 \\
\Rightarrow \tilde w &= (X^TX+2 \lambda I)^{-1}X^Ty \\ 
\end{align} 

__Norm Penalties OR Constrained Optimization:__

We can thus think of a parameter norm penalty as imposing a constraint on the weights. If Ω is the L2 norm, then the weights are constrained to lie in an L2 ball. If Ω is the L1 norm, then the weights are constrained to lie in a region of limited L1 norm.

Sometimes we may wish to use explicit constraints rather than penalties because penalties can cause non-convex optimization procedures to get __stuck in local minima__ corresponding to small θ.

When training neural networks, this usually manifests as neural networks that train with several __dead units__. These are units that do not contribute much to the behavior of the function learned by the network because __the weights going into or out of them are all very small.__ 

When training with a penalty on the norm of the weights, these configurations can be locally optimal, even if it is possible to significantly reduce J by making the weights larger. Explicit constraints implemented by re-projection can work much better in these cases because they do not encourage the weights to approach the origin. Explicit constraints implemented by re-projection only have an effect when the weights become large and attempt to leave the constraint region.

Finally, explicit constraints with reprojection can be useful because they impose some stability on the optimization procedure. When using high learning rates, it is possible to encounter a positive feedback loop in which large weights induce large gradients which then induce a large update to the weights. If these updates consistently increase the size of the weights, then θ rapidly moves away from the origin until numerical overflow occurs. Explicit constraints with reprojection prevent this feedback loop from continuing to increase the magnitude of the weights
without bound. Hinton et al. (2012c) recommend using constraints combined with a high learning rate to allow rapid exploration of parameter space while maintaining some stability.

In particular, Hinton et al. (2012c) recommend a strategy introduced by Srebro and Shraibman (2005): constraining the norm of each column of the weight matrix of a neural net layer, rather than constraining the Frobenius norm of the entire weight matrix. Constraining the norm of each column separately prevents any one hidden unit from having very large weights. If we converted this constraint into a penalty in a Lagrange function, it would be similar to L2 weight decay but with a separate KKT multiplier for the weights of each hidden unit. Each of these KKT multipliers would be dynamically updated separately to make each hidden unit obey the constraint. In practice, column norm limitation is always implemented as an explicit constraint with reprojection.